In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
nltk.download('wordnet')
nltk.download('stopwords')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

In [2]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [3]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [5]:
sw = stopwords.words('english')
sw.append(['input', 'output', 'th'])
lemma = WordNetLemmatizer()
text = data.problem_statement

# Problem_statement
text = text.apply(lambda x: re.sub('[,|.|(|)|$|!|?|!]',' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo

# Topics
topics = data.problem_tags
topics = topics.apply(lambda x: re.sub('[*][0-9]+','',x)) # remove os ratings *800 etc
topics = topics.apply(lambda x: [i for i in x.split(',') if i != '']) # tira strings vazias

In [6]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    n block arranged row numbered left right start...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let's define function f x follows: \begi...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

In [7]:
topics

0                                        [implementation]
1                                    [binarysearch, math]
2                  [dfsandsimilar, graphs, shortestpaths]
3                                                   [dsu]
4       [combinatorics, dfsandsimilar, dsu, graphs, gr...
                              ...                        
8338                                       [greedy, math]
8339       [bruteforce, geometry, greedy, implementation]
8340    [datastructures, dp, greedy, implementation, s...
8341              [binarysearch, combinatorics, dp, math]
8342                                         [bruteforce]
Name: problem_tags, Length: 6819, dtype: object

## Vertorizando

In [8]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df, ngram_range=(1,3))

In [9]:
X = vec.fit_transform(text)
X.shape

(6819, 399)

## Agrupando

In [10]:
# Pegando quantidade de topicos

total_topics = set()
topics = topics.apply(lambda x: [total_topics.add(i) for i in x])

In [11]:
n_total_topics = len(total_topics)
print(f'Número de Tópicos: {n_total_topics}')
total_topics

Número de Tópicos: 37


{'*specialproblem',
 '2-sat',
 'binarysearch',
 'bitmasks',
 'bruteforce',
 'chineseremaindertheorem',
 'combinatorics',
 'constructivealgorithms',
 'datastructures',
 'dfsandsimilar',
 'divideandconquer',
 'dp',
 'dsu',
 'expressionparsing',
 'fft',
 'flows',
 'games',
 'geometry',
 'graphmatchings',
 'graphs',
 'greedy',
 'hashing',
 'implementation',
 'interactive',
 'math',
 'matrices',
 'meet-in-the-middle',
 'numbertheory',
 'probabilities',
 'schedules',
 'shortestpaths',
 'sortings',
 'strings',
 'stringsuffixstructures',
 'ternarysearch',
 'trees',
 'twopointers'}

In [12]:
n_components = n_total_topics
max_iter = 100

lda = LatentDirichletAllocation(n_components=n_components, max_iter=max_iter)

In [13]:
lda_matrix = lda.fit_transform(X)

In [14]:
top_word = lda.components_
terms = vec.get_feature_names_out()

for i, comp in enumerate(top_word):
    top_terms_key = zip(terms, comp)
    top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
    top_terms_list= list(dict(top_terms_key).keys())
    print("Topic "+str(i)+": ",top_terms_list)

Topic 0:  ['letter', 'lowercase', 'empty', 'english', 'non empty', 'non', 'length', 'consisting', 'one', 'line contains']
Topic 1:  ['string', 'character', 'length', 'consisting', 'string length', 'contains string', 'example', 'given', 'string consisting', 'line contains']
Topic 2:  ['city', 'segment', 'path', 'connected', 'one', 'two', 'length', 'pair', 'possible', 'way']
Topic 3:  ['example', 'maximum', 'second', 'possible', 'one', 'first example', 'second example', 'third', 'total', 'line contains']
Topic 4:  ['100', 'le 100', '100 number', 'le le 100', 'follow', 'result', 'consisting', 'zero', 'initial', 'also']
Topic 5:  ['test', 'case', 'test case', 'first test', 'second', 'second test', 'first test case', 'second test case', 'third', 'one']
Topic 6:  ['single', 'single integer', 'contains single', 'contains single integer', 'line contains single', 'line contains', 'print single', 'next', 'next line', 'output single']
Topic 7:  ['operation', 'minimum', 'minimum number', 'perform'

## Validando

In [15]:
# todo

## Visualizando

In [16]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda, X, vec, mds='tsne')
panel
# todo

/home/anasofia/Documents/UnB/PIBIC/classificador-cf/env/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -3.142260  1.263473       1        1  6.051947
26    -2.673595  1.961177       2        1  5.855574
19    -2.709292  2.042684       3        1  5.478379
32    -2.859059  2.219482       4        1  5.409351
3     -2.798695  1.916706       5        1  5.346911
33    -2.699766  1.451892       6        1  5.281228
17    -2.708857  1.850719       7        1  4.717158
27    -3.594802  1.633080       8        1  4.285852
12    -3.267340  1.353585       9        1  3.539829
8     -2.401114  1.714845      10        1  3.271588
23    -2.691694  1.323192      11        1  3.227559
29    -3.158109  1.053869      12        1  3.223245
1     -2.013686  1.946856      13        1  3.060163
18    -3.272519  1.802088      14        1  2.900130
35    -2.498890  2.257585      15        1  2.732494
10    -3.009676  1.351463      16        1  2.646494
7     -2.961066  2.551337      17        1  2.441936
16    -2.571857  1.117544      18        1  2.309838
6     -2.520943  0.934696      19        1  2.212238
36    -2.255639  1.709370      20        1  2.138499
15    -2.668676  1.915473      21        1  2.131545
24    -1.832342  1.770245      22        1  1.885949
9     -2.748863  1.941167      23        1  1.861878
2     -2.741068  2.275675      24        1  1.847341
5     -3.525495  1.860222      25        1  1.728359
31    -2.295203  2.763297      26        1  1.712479
20    -1.861641  1.566587      27        1  1.590399
0     -2.111785  2.120408      28        1  1.521625
34    -2.560039  2.665912      29        1  1.493813
28    -1.835791  1.455131      30        1  1.458752
22    -1.815889  1.931307      31        1  1.413038
25    -2.945591  2.582275      32        1  1.384820
21    -3.014905  2.336057      33        1  1.033356
11    -3.155926  2.953732      34        1  0.980932
30    -3.577348  2.563265      35        1  0.807102
4     -3.966392  2.005085      36        1  0.601603
14    -1.867715  2.432373      37        1  0.416599, topic_info=                  Term          Freq         Total Category  logprob  loglift
190                 le  13554.000000  13554.000000  Default  30.0000  30.0000
348             string   6990.000000   6990.000000  Default  29.0000  29.0000
354               test   6551.000000   6551.000000  Default  28.0000  28.0000
326           sequence   4394.000000   4394.000000  Default  27.0000  27.0000
207                leq   4556.000000   4556.000000  Default  26.0000  26.0000
..                 ...           ...           ...      ...      ...      ...
254  next line contain      4.061684    850.938099  Topic37  -6.4532   0.1361
344              start      5.979071   1303.724075  Topic37  -6.0665   0.0961
265                one     37.878343  13620.025574  Topic37  -4.2204  -0.4041
315             second     28.001683   9438.764580  Topic37  -4.5225  -0.3395
122            example     18.888678   5673.745321  Topic37  -4.9162  -0.2242

[1481 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.043139   000
0         5  0.191650   000
0         7  0.082742   000
0        10  0.025459   000
0        11  0.022630   000
...     ...       ...   ...
398      25  0.019874  zero
398      27  0.022082  zero
398      31  0.022818  zero
398      34  0.229653  zero
398      36  0.000736  zero

[2906 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 27, 20, 33, 4, 34, 18, 28, 13, 9, 24, 30, 2, 19, 36, 11, 8, 17, 7, 37, 16, 25, 10, 3, 6, 32, 21, 1, 35, 29, 23, 26, 22, 12, 31, 5, 15])